In [244]:
import random
import re
from urllib.parse import urlparse, parse_qs, parse_qsl, quote

import pandas as pd
import requests

from tqdm import tqdm

from bs4 import BeautifulSoup
import bs4

URL_ROOT = 'https://en.wiktionary.org'

soups = {}
def get_soup(path):
    if path in soups:
        return soups[path]
    
    res = requests.get(f"{URL_ROOT}{path}")
    soup = BeautifulSoup(res.text)
    soups[path] = soup
    return soup

# e.g. get_category_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_category_pages(start_path):
    paths = [start_path]    
    next_path = start_path
    soup = get_soup(next_path)
    next_link = soup.find('a', string='next page')
    while next_link:
        next_path = next_link['href']
        paths.append(next_path)
        soup = get_soup(next_path)
        next_link = soup.find('a', string='next page')
    return paths

# e.g. get_lemma_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_lemma_pages(path):
    soup = get_soup(path)
    return [{'href': a['href'], 'title': a['title']} for a in soup.css.select('div#mw-pages li a')]

In [245]:
pd.set_option('display.max_colwidth', 250)

In [246]:
# Category:Armenian_terms_with_usage_examples
# Category:Old_Armenian_terms_with_usage_examples

lps = ([lp for cp in get_category_pages('/wiki/Category:Armenian_terms_with_usage_examples')
    for lp in get_lemma_pages(cp)])
lemmas = pd.DataFrame(lps)
lemmas

,href,title
0,/wiki/%D5%A1,ա
1,/wiki/%D5%A1%CC%88,ա̈
2,/wiki/%D5%A1%CC%8A,ա̊
3,/wiki/%D5%A1%D5%A2%D5%A5%D5%A9%D5%A1%D5%BD%D5%B8%D6%82%D5%B6%D5%AF,աբեթասունկ
4,/wiki/%D5%A1%D5%A3%D5%BC%D5%A1%D5%BE,ագռավ
...,...,...
1788,/wiki/%D5%96%D6%80%D5%B8%D6%82%D5%B6%D5%A6%D5%A5,Ֆրունզե
1789,/wiki/%D5%9B,՛
1790,/wiki/%D5%9E,՞
1791,/wiki/%D6%89,։


In [247]:
def getLanguageSections(soup):
    headings = soup.select('div#bodyContent div.mw-heading2')
    
    heading_map = {}

    for heading in headings:
        language = heading.find('h2').text
        content = []
        for sib in heading.next_siblings:
            if sib == '\n':
                continue
            if type(sib) == bs4.element.Comment:
                continue
            if sib in headings:
                break
            content.append(sib)
        heading_map[language] = content
    return heading_map

In [248]:
def getHeadingLevel(cs):
    levels = [c for c in cs if c != 'mw-heading']
    if len(levels) > 0:
        return levels[0]

def getSubSections(section):
    results = []
    result = {}
    current_header = None
    current_subsection = bs4.Tag(name='div')
    top_heading_level = None
    for tag in section:
        classes = tag.attrs.get('class', [])
        if tag.name == 'div' and 'mw-heading' in classes:
            heading_level = getHeadingLevel(classes)
            if not top_heading_level:
                top_heading_level = heading_level
            elif heading_level == top_heading_level and result:
                # the first time you encounter the top-level heading again...
                if len(result) > 1: # this means there were other headings in between
                    results.append(result)
                    result = {}
                else: # consecutive top-level headings means flat hierarchy
                    top_heading_level = '_' # non-existent heading
            if current_header:
                result[current_header] = current_subsection
            current_header = tag.text.replace('[edit]', '')
            current_subsection = bs4.Tag(name='div')
        else:
            # if not heading, add to current heading's subsection 
            current_subsection.append(tag)
    if current_header:
        result[current_header] = current_subsection
    results.append(result)
    return results

In [371]:
entries = {}

for title, href in tqdm([row[['title','href']].to_list() for _, row in lemmas.sample(500).iterrows()]):
#for title, href in tqdm([row[['title','href']].to_list() for _, row in lemmas[lemmas.title.isin(['աման', 'փիս', '-անք', 'համ'])].iterrows()]):
    sections = getLanguageSections(get_soup(href))
    if 'Armenian' not in sections:
        continue
    entries[title] = getSubSections(sections['Armenian'])

100%|██████████| 500/500 [07:39<00:00,  1.09it/s]


In [372]:
results = pd.DataFrame([dict(title=k, idx=i+1, **r) for k,v in entries.items() for i,r in enumerate(v)]).set_index(['title', 'idx'])

In [373]:
results

Etymology  \
title          idx                                                                                                                                                                                                                                                              
մատաղ          1         [[From , [<a class="extiw" href="https://en.wikipedia.org/wiki/Classical_Armenian" title="w:Classical Armenian">Old Armenian</a>],  , [<a class="mw-selflink-fragment" href="#Old_Armenian">մատաղ</a>],  , [(], [matał], [)], ; see it for more.\n]]   
արդարություն   1    [[From , [<a class="extiw" href="https://en.wikipedia.org/wiki/Classical_Armenian" title="w:Classical Armenian">Old Armenian</a>],  , [<a href="/wiki/%D5%A1%D6%80%D5%A4%D5%A1%D6%80%D5%B8%D6%82%D5%A9%D5%AB%D6%82%D5%B6#Old_Armenian" title="արդարութ...   
սիրահար        1    [[Literally, "love-stricken", from , [<a href="/wiki/%D5%BD%D5%A5%D6%80#Armenian" title="սեր">սեր</a>],  , [(], [ser], [)],  +‎ , [<a href="/wiki/-%D5%A1-#Armenian" title="-ա-">-ա-</a>],  , [(], [-a-], [)],  +‎ , [<a class="extiw" href="https://e...   
այս            1                                                                                                                                                                                                                                                          NaN   
               2                                                                                                                                                                                                                                                          NaN   
...                                                                                                                                                                                                                                                                       ...   
համատիրություն 1    [[[<a href="/wiki/%D5%B0%D5%A1%D5%B4%D5%A1-#Armenian" title="համա-">համա-</a>],  , [(], [hama-], [)],  +‎ , [<a href="/wiki/%D5%BF%D5%AB%D6%80%D5%B8%D6%82%D5%A9%D5%B5%D5%B8%D6%82%D5%B6#Armenian" title="տիրություն">տիրություն</a>],  , [(], [tirutʻ...   
թառ            1                                                                                                                                                                                                                                                          NaN   
               2                                                                                                                                                                                                                                                          NaN   
օրհներգ        1                                        [[[<a href="/wiki/%D6%85%D6%80%D5%B0%D5%B6%D5%A5%D5%AC#Armenian" title="օրհնել">օրհնել</a>],  , [(], [ōrhnel], [)],  +‎ , [<a href="/wiki/%D5%A5%D6%80%D5%A3#Armenian" title="երգ">երգ</a>],  , [(], [erg], [)], \n]]   
կարոտ          1     [[From , [<a class="extiw" href="https://en.wikipedia.org/wiki/Classical_Armenian" title="w:Classical Armenian">Old Armenian</a>],  , [<a href="/wiki/%D5%AF%D5%A1%D6%80%D6%85%D5%BF#Old_Armenian" title="կարօտ">կարօտ</a>],  , [(], [karōt], [)], .\n]]   

                                                                                                                                                                                                                                                                Pronunciation  \
title          idx                                                                                                                                                                                                                                                              
մատաղ          1    [[[], \n, [<span class="ib-brac qualifier-brac">(</span>, <span class="ib-content qualifier-content"><span class="usage-label-accent"><a class="

In [374]:
columns_by_freq = results.count().sort_values(ascending=False)
results = results[columns_by_freq.index] #[lambda x:x['Verb'].notna()]
columns_by_freq.to_frame().style.bar()

,0
Pronunciation,496
Etymology,446
Declension,401
Noun,301
Alternative forms,154
Adjective,137
Derived terms,126
References,74
Related terms,64
Verb,56


In [375]:
def process_pronunciation(tag):
    if type(tag) != bs4.Tag:
        return tag
    
    return '\n'.join([li.text for li in tag.select('li') if not 'Audio' in li.text and not li.text == ''])

results['Pronunciation'] = results['Pronunciation'].apply(process_pronunciation)

In [376]:
def process_basic_text(tag):
    if type(tag) != bs4.Tag:
        return tag
    
    return tag.text.strip()


results['Etymology'] = results['Etymology'].apply(process_basic_text)
results['Etymology'] = results['Etymology 1'].apply(process_basic_text).where(results['Etymology 1'].notna(), results['Etymology'])
results['Etymology'] = results['Etymology 2'].apply(process_basic_text).where(results['Etymology 2'].notna(), results['Etymology'])

In [377]:
parts_of_speech = ['Verb',
 'Adverb',
 'Particle',
 'Pronoun',
 'References',
 'Suffix',
 'Proper noun',
 'Conjunction',
 'Postposition',
 'Interjection',
 'Preposition',
 'Determiner',
 'Punctuation mark',
 'Numeral',
 'Prefix',
 'Article']

In [387]:
results[parts_of_speech].count(axis=1)[lambda x: x==0]

title   idx
այս     1      0
մը      1      0
հանուն  1      0
ավտո-   1      0
ընդդեմ  1      0
աման    2      0
այդ     1      0
վար     3      0
ինձ     1      0
dtype: int64

In [390]:
def process_definition(tag):
    if type(tag) != bs4.Tag:
        return tag
    definitions = []
    if not tag.find('ol'):
        return [{'definition': 'ERROR!!!!'}]
    for li in tag.find('ol').find_all('li'):
        if li.find('dl'):
            detail_text = li.find('dl').text
            definitions.append({'definition': li.text.replace(detail_text, '').strip(), 'detail': li.find('dl').text})
        else:
            definitions.append({'definition': li.text})
    return definitions


In [403]:
pos = 'Verb'
results[results[pos].notna()][pos].sample(10).apply(process_definition).apply(pd.Series).stack().apply(pd.Series).fillna('')

definition  \
title       idx                                                                            
մատակարարել 1   0                          (transitive) to provide, to supply, to purvey   
փայփայել    1   0                                                   to caress, to stroke   
                1                                      to show affection, fondness, care   
                2                   (figuratively) to cherish, to treasure, to hold dear   
ծառայել     1   0                                                               to serve   
խոսել       1   0                                                               to speak   
                1                                                                to talk   
մթնել       1   0                  (intransitive) to get dark, to become dark, to darken   
արորել      1   0                                                              to plough   
սովորել     1   0                                                (intransitive) to study   
                1                                                  (transitive) to learn   
                2                        (intransitive) to become accustomed, habituated   
սթափվել     1   0                                     to sober up (especially from wine)   
                1  to come to oneself, to come to one's senses, to pull oneself together   
                2                                       to focus, to heed, pay attention   
                3                                       to bring back to life, to revive   
                4                               to wake up, to invigorate, to revitalize   
ցավել       1   0                                                       to ache, to hurt   
                1                                                              to regret   
                2                     to be a fan of, to support (usually a sports team)   
վերաբերվել  1   0                                                              ERROR!!!!   

                                                                                                                                                                                                                                                                      detail  
title       idx                                                                                                                                                                                                                                                               
մատակարարել 1   0  Մեկնարկել է Ռուսաստանից Հայաստանին մատակարարվող գազի գնի շուրջ բանակցությունների հերթական փուլը ― Meknarkel ē Ṙusastanicʻ Hayastanin matakararvoġ gazi gni šurǰ banakcʻutʻyunneri hertʻakan pʻulə ― Another round of negotations on the price of gas s...  
փայփայել    1   0                                                                                                                                                                                                                                                             
                1                                                                                                                                                                                                                                                             
                2                                                                                                                                                                                                    հույսեր փայփայել ― huyser pʻaypʻayel ― to cherish hopes  
ծառայել     1   0                                                                                                                                                                                                  բանակում ծառայել ― banakum caṙayel ― to serve in the army  
խոսել       1   0                           բարձր խոսել ― barjr 

In [380]:
def find_element(tag, element):
    if type(tag) != bs4.Tag:
        return tag
    return tag.find(element)

In [381]:
(pd.DataFrame({col: {elem: results[col].apply(lambda x:find_element(x,elem)).count()  for elem in ['ol', 'ul']} for col in results.columns}).T['ol']
 [lambda x:x>0][lambda x:x<90].sort_values(ascending=False).index.to_list()
)

['Verb',
 'Adverb',
 'Particle',
 'Pronoun',
 'References',
 'Suffix',
 'Proper noun',
 'Conjunction',
 'Postposition',
 'Interjection',
 'Preposition',
 'Determiner',
 'Punctuation mark',
 'Numeral',
 'Prefix',
 'Article']